# 03b - Validazione e Pulizia Dataset

Questo notebook gestisce la sincronizzazione tra i CSV e le immagini effettivamente presenti dopo la revisione manuale.

**Operazioni eseguite:**
1. Scansione delle cartelle con immagini preprocessate
2. Confronto con i CSV esistenti
3. Rimozione dai CSV delle righe relative a immagini eliminate
4. Backup dei CSV originali
5. Report delle immagini rimosse

## 1. Import e Configurazione

In [ ]:
import os
import sys
from pathlib import Path
from datetime import datetime
import shutil

# Manipolazione dati
import pandas as pd
import numpy as np

# Configurazione
import yaml

print(f"Esecuzione: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Funzione per trovare la root del progetto
def get_project_root():
    """Rileva la root del progetto."""
    notebook_dir = Path.cwd()
    if notebook_dir.name == 'notebooks':
        return notebook_dir.parent
    current = notebook_dir
    for _ in range(5):
        if (current / 'Data').exists():
            return current
        current = current.parent
    # Fallback per WSL
    if sys.platform == 'linux' and Path('/mnt/c').exists():
        return Path('/mnt/c/Repository/multi-method-xai-diabetic-retinopathy')
    return Path('C:/Repository/multi-method-xai-diabetic-retinopathy')

PROJECT_ROOT = get_project_root()
print(f"Project root: {PROJECT_ROOT}")

In [ ]:
# Caricamento configurazione
config_path = PROJECT_ROOT / 'config.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Setup percorsi
PROCESSED_DIR = PROJECT_ROOT / config['paths']['processed_dir']
PREPROCESSED_DIR = PROJECT_ROOT / config['paths']['preprocessed_images']
BACKUP_DIR = PROCESSED_DIR / 'backup_csv'

print(f"Directory processed: {PROCESSED_DIR}")
print(f"Directory immagini: {PREPROCESSED_DIR}")

## 2. Scansione Immagini Esistenti

In [ ]:
def scan_existing_images(images_dir):
    """
    Scansiona la directory delle immagini preprocessate e restituisce
    un set con tutti i nomi dei file esistenti.
    
    Args:
        images_dir: Path alla directory delle immagini
        
    Returns:
        dict: Dizionario {nome_file: percorso_completo}
    """
    existing_images = {}
    extensions = {'.png', '.jpg', '.jpeg'}
    
    for root, dirs, files in os.walk(images_dir):
        for file in files:
            if Path(file).suffix.lower() in extensions:
                full_path = Path(root) / file
                # Salva con percorso relativo dalla root del progetto
                rel_path = full_path.relative_to(PROJECT_ROOT)
                existing_images[file] = str(rel_path)
    
    return existing_images

# Scansiona le immagini
print("Scansione immagini in corso...")
existing_images = scan_existing_images(PREPROCESSED_DIR)
print(f"Immagini trovate: {len(existing_images):,}")

In [ ]:
# Mostra alcuni esempi di immagini trovate
print("\nEsempi di immagini trovate:")
for i, (name, path) in enumerate(list(existing_images.items())[:5]):
    print(f"  {name} -> {path}")

## 3. Validazione CSV

In [ ]:
def validate_csv(csv_path, existing_images, project_root):
    """
    Valida un CSV confrontandolo con le immagini esistenti.
    
    Args:
        csv_path: Path al file CSV
        existing_images: Dict delle immagini esistenti
        project_root: Root del progetto
        
    Returns:
        tuple: (df_valid, df_removed, stats)
    """
    df = pd.read_csv(csv_path)
    original_count = len(df)
    
    # Estrai il nome del file dalla colonna preprocessed_path
    def get_filename(path):
        return Path(path).name
    
    df['_filename'] = df['preprocessed_path'].apply(get_filename)
    
    # Verifica quali immagini esistono
    df['_exists'] = df['_filename'].apply(lambda x: x in existing_images)
    
    # Separa righe valide e rimosse
    df_valid = df[df['_exists']].copy()
    df_removed = df[~df['_exists']].copy()
    
    # Aggiorna i percorsi con quelli corretti (nel caso siano cambiati)
    df_valid['preprocessed_path'] = df_valid['_filename'].apply(
        lambda x: existing_images.get(x, '')
    )
    
    # Rimuovi colonne temporanee
    df_valid = df_valid.drop(columns=['_filename', '_exists'])
    df_removed = df_removed.drop(columns=['_exists'])
    
    # Statistiche
    stats = {
        'original': original_count,
        'valid': len(df_valid),
        'removed': len(df_removed),
        'pct_removed': len(df_removed) / original_count * 100 if original_count > 0 else 0
    }
    
    return df_valid, df_removed, stats

In [ ]:
# Lista dei CSV da validare
csv_files = [
    'aptos_preprocessed.csv',
    'eyepacs_preprocessed.csv',
    'messidor2_preprocessed.csv',
    'ddr_preprocessed.csv',
    'combined_train_preprocessed.csv'
]

# Verifica quali CSV esistono
existing_csvs = [f for f in csv_files if (PROCESSED_DIR / f).exists()]
print(f"CSV trovati: {len(existing_csvs)}")
for csv in existing_csvs:
    print(f"  - {csv}")

## 4. Backup e Aggiornamento CSV

In [ ]:
# Crea directory di backup
BACKUP_DIR.mkdir(exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

print(f"Directory backup: {BACKUP_DIR}")
print(f"Timestamp: {timestamp}")

In [ ]:
# Dizionario per raccogliere tutte le statistiche
all_stats = {}
all_removed = []

for csv_file in existing_csvs:
    csv_path = PROCESSED_DIR / csv_file
    print(f"\n{'='*60}")
    print(f"Elaborazione: {csv_file}")
    print('='*60)
    
    # Validazione
    df_valid, df_removed, stats = validate_csv(csv_path, existing_images, PROJECT_ROOT)
    all_stats[csv_file] = stats
    
    print(f"  Righe originali: {stats['original']:,}")
    print(f"  Righe valide: {stats['valid']:,}")
    print(f"  Righe rimosse: {stats['removed']:,} ({stats['pct_removed']:.2f}%)")
    
    if stats['removed'] > 0:
        # Backup del CSV originale
        backup_path = BACKUP_DIR / f"{csv_file.replace('.csv', '')}_{timestamp}.csv"
        shutil.copy(csv_path, backup_path)
        print(f"  Backup salvato: {backup_path.name}")
        
        # Salva CSV aggiornato
        df_valid.to_csv(csv_path, index=False)
        print(f"  CSV aggiornato salvato")
        
        # Aggiungi al report
        df_removed['source_csv'] = csv_file
        all_removed.append(df_removed)
        
        # Mostra alcune immagini rimosse
        print(f"\n  Esempi di immagini rimosse:")
        for _, row in df_removed.head(5).iterrows():
            print(f"    - {row['_filename']}")
    else:
        print(f"  Nessuna modifica necessaria")

## 5. Report Finale

In [ ]:
# Riepilogo generale
print("\n" + "="*60)
print("RIEPILOGO VALIDAZIONE")
print("="*60)

total_original = sum(s['original'] for s in all_stats.values())
total_valid = sum(s['valid'] for s in all_stats.values())
total_removed = sum(s['removed'] for s in all_stats.values())

print(f"\nImmagini nel filesystem: {len(existing_images):,}")
print(f"\nRighe totali nei CSV:")
print(f"  Prima: {total_original:,}")
print(f"  Dopo: {total_valid:,}")
print(f"  Rimosse: {total_removed:,}")

if total_removed > 0:
    print(f"\nPercentuale rimossa: {total_removed/total_original*100:.2f}%")

In [ ]:
# Tabella riepilogativa per CSV
print("\nDettaglio per CSV:")
print("-" * 70)
print(f"{'CSV':<35} {'Originali':>10} {'Valide':>10} {'Rimosse':>10}")
print("-" * 70)

for csv_file, stats in all_stats.items():
    print(f"{csv_file:<35} {stats['original']:>10,} {stats['valid']:>10,} {stats['removed']:>10,}")

print("-" * 70)
print(f"{'TOTALE':<35} {total_original:>10,} {total_valid:>10,} {total_removed:>10,}")

In [ ]:
# Salva report delle immagini rimosse
if all_removed:
    df_all_removed = pd.concat(all_removed, ignore_index=True)
    report_path = PROCESSED_DIR / f'removed_images_report_{timestamp}.csv'
    df_all_removed.to_csv(report_path, index=False)
    print(f"\nReport immagini rimosse salvato: {report_path.name}")
    print(f"Totale immagini nel report: {len(df_all_removed):,}")
else:
    print("\nNessuna immagine rimossa - nessun report generato")

In [ ]:
# Distribuzione classi dopo la pulizia (se ci sono state rimozioni)
if total_removed > 0:
    print("\nDistribuzione classi dopo la pulizia:")
    print("-" * 50)
    
    class_names = config['model']['class_names']
    
    # Ricarica il combined train aggiornato
    combined_path = PROCESSED_DIR / 'combined_train_preprocessed.csv'
    if combined_path.exists():
        df_train = pd.read_csv(combined_path)
        print(f"\nTraining set ({len(df_train):,} immagini):")
        
        class_counts = df_train['diagnosis'].value_counts().sort_index()
        for cls, count in class_counts.items():
            pct = count / len(df_train) * 100
            print(f"  Classe {cls} ({class_names[cls]}): {count:,} ({pct:.1f}%)")

## 6. Ricalcolo Class Weights

In [ ]:
# Se ci sono state modifiche, ricalcola i class weights
if total_removed > 0:
    import torch
    
    combined_path = PROCESSED_DIR / 'combined_train_preprocessed.csv'
    if combined_path.exists():
        df_train = pd.read_csv(combined_path)
        labels = df_train['diagnosis'].values
        
        # Calcola nuovi class weights
        class_counts = np.bincount(labels)
        n_samples = len(labels)
        n_classes = len(class_counts)
        weights = n_samples / (n_classes * class_counts)
        class_weights = torch.FloatTensor(weights)
        
        # Backup dei vecchi weights
        old_weights_path = PROCESSED_DIR / 'class_weights.pt'
        if old_weights_path.exists():
            backup_weights = BACKUP_DIR / f'class_weights_{timestamp}.pt'
            shutil.copy(old_weights_path, backup_weights)
            print(f"Backup class weights: {backup_weights.name}")
        
        # Salva nuovi weights
        torch.save(class_weights, old_weights_path)
        
        print("\nNuovi class weights calcolati:")
        for i, w in enumerate(class_weights):
            print(f"  Classe {i} ({class_names[i]}): {w:.4f}")
else:
    print("Nessuna modifica ai CSV - class weights invariati")

## 7. Conclusioni

### Operazioni Completate

1. **Scansione filesystem**: Identificate tutte le immagini preprocessate esistenti
2. **Validazione CSV**: Confronto tra riferimenti nei CSV e immagini reali
3. **Backup**: Salvati i CSV originali prima delle modifiche
4. **Aggiornamento**: Rimossi dai CSV i riferimenti a immagini eliminate
5. **Report**: Generato elenco delle immagini rimosse
6. **Class Weights**: Ricalcolati i pesi per il training

### File Generati

- `backup_csv/`: Directory con i CSV originali
- `removed_images_report_YYYYMMDD_HHMMSS.csv`: Elenco immagini rimosse
- `class_weights.pt`: Pesi aggiornati per il training